In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor


In [2]:
BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.001




In [3]:
class FeedForwardNet(nn.Module):   # create a class that inherits from Module (nn.Module)
    # For defining a model in pytorch we need to define a couple of methods: 1) constructor and 2) forward
    
    def __init__(self):
        super().__init__()  #invoke the constructor of Module class
                            # store different layers we need for the model as attributes in this class
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_data):    #defining Forward method: indicates to pytorch how manipulate data and in what sequence
        flattened_data = self.flatten(input_data)
        logits = self.dense_layers(flattened_data)
        predictions = self.softmax(logits)
        return predictions


In [4]:

def download_mnist_datasets():
    train_data = datasets.MNIST( # MNIST: dataset class that comes with pyTorch
        root = "data",            #where to store the data
        download = True,
        train = True,
        transform = ToTensor()  # transforms allows to apply some sort of a transform to a dataset, in this case 
                                # we are applying ToTensor: takes an image in and reshapes to a tensor where values 
                                # are normalized between 0 and 1
    )
    validation_data = datasets.MNIST( # MNIST: dataset class that comes with pyTorch
        root = "data",            #where to store the data
        download = True,
        train = False,
        transform = ToTensor()  # transforms allows to apply some sort of a transform to a dataset, in this case 
                                # we are applying ToTensor: takes an image in and reshapes to a tensor where values 
                                # are normalized between 0 and 1
    )
    return train_data, validation_data

In [5]:
# Data Loader: Class that we can use to wrap a dataset that will allow us to fetch data in batches

# Creating a data-loader
def create_data_loader(train_data, batch_size):
    train_data_loader = DataLoader(train_data, batch_size=batch_size)
    return train_data_loader


In [6]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
    for inputs, targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)    #need to do this for Pytorch
        
        # Calculate loss
        predictions = model(inputs)
        loss = loss_fn(predictions, targets)
        
        
        # Backpropagate loss and update weights
        optimiser.zero_grad()    # reset the gradients to zero 
        loss.backward()  #apply backpropagation
        optimiser.step()  # updating weights
        
    print(f"Loss: {loss.item()}")

def train(model, data_loader, loss_function, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_one_epoch(model, data_loader, loss_fn, optimiser, device)
        print("----------------------------")
    print("Training is done")    
    

In [7]:
if __name__ == "__main__":
    #download MNIST dataset
    train_data, validation_data = download_mnist_datasets()
    print("MNIST dataset downloaded")
    


MNIST dataset downloaded


In [8]:
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
# Build a model
# Models that you create in PyTorch are in Classes, which inherit from the module CLASS, that comes driectly from Pytorch

feed_forward_net = FeedForwardNet().to(device)


In [11]:
# instantiate loss function and optimiser
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(feed_forward_net.parameters(),
                            lr=LEARNING_RATE)


In [12]:
train_data_loader = create_data_loader(train_data, BATCH_SIZE)


In [13]:
train_data_loader = create_data_loader(train_data, BATCH_SIZE)
train(feed_forward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

Epoch 1
Loss: 1.5360289812088013
----------------------------
Epoch 2
Loss: 1.4963124990463257
----------------------------
Epoch 3
Loss: 1.488261342048645
----------------------------
Epoch 4
Loss: 1.4838236570358276
----------------------------
Epoch 5
Loss: 1.4779716730117798
----------------------------
Epoch 6
Loss: 1.4754638671875
----------------------------
Epoch 7
Loss: 1.4740115404129028
----------------------------
Epoch 8
Loss: 1.4734076261520386
----------------------------
Epoch 9
Loss: 1.4728575944900513
----------------------------
Epoch 10
Loss: 1.4727519750595093
----------------------------
Training is done


In [14]:
torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
print("Model trained and stored at feedforwardnet.pth")

Model trained and stored at feedforwardnet.pth
